In [1]:
import pandas as pd 
import json

In [2]:
#set max display in notebook
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [3]:
json_file_path = 'cb_2021_us_tract_500k.geojson'

In [4]:
# Load GeoJSON data from the file
with open(json_file_path, 'r') as file:
    geojson_data = json.load(file)

In [5]:
# Extract features from the GeoJSON data
features = geojson_data['features']

In [6]:
# Create a list to store the data
data_list = []

In [7]:
# Iterate through features and extract properties and geometry
for feature in features:
    properties = feature['properties']
    geometry = feature['geometry']
    
    # Combine properties and geometry into a single dictionary
    row_data = {**properties, 'geometry': json.dumps(geometry)}
    
    # Append the dictionary to the list
    data_list.append(row_data)

In [8]:
df = pd.DataFrame(data_list)
df.shape
df.head()

(85230, 14)

,STATEFP,COUNTYFP,TRACTCE,AFFGEOID,GEOID,NAME,NAMELSAD,STUSPS,NAMELSADCO,STATE_NAME,LSAD,ALAND,AWATER,geometry
0,13,163,960300,1400000US13163960300,13163960300,9603,Census Tract 9603,GA,Jefferson County,Georgia,CT,564728206,4845400,"{""type"": ""Polygon"", ""coordinates"": [[[-82.6619..."
1,37,109,071201,1400000US37109071201,37109071201,712.01,Census Tract 712.01,NC,Lincoln County,North Carolina,CT,5976613,6611763,"{""type"": ""Polygon"", ""coordinates"": [[[-80.9934..."
2,25,025,050200,1400000US25025050200,25025050200,502,Census Tract 502,MA,Suffolk County,Massachusetts,CT,243271,0,"{""type"": ""Polygon"", ""coordinates"": [[[-71.0400..."
3,17,031,230200,1400000US17031230200,17031230200,2302,Census Tract 2302,IL,Cook County,Illinois,CT,214283,0,"{""type"": ""Polygon"", ""coordinates"": [[[-87.7134..."
4,41,047,001607,1400000US41047001607,41047001607,16.07,Census Tract 16.07,OR,Marion County,Oregon,CT,1814790,0,"{""type"": ""Polygon"", ""coordinates"": [[[-122.990..."


In [9]:
for col in df.columns:
    print(col)

STATEFP
COUNTYFP
TRACTCE
AFFGEOID
GEOID
NAME
NAMELSAD
STUSPS
NAMELSADCO
STATE_NAME
LSAD
ALAND
AWATER
geometry


In [10]:
df['Census Tract ID'] = df['NAMELSAD'].str.replace('Census Tract ', '')
df = df.drop(columns=['NAMELSAD'])
df.shape
df.head()

(85230, 14)

,STATEFP,COUNTYFP,TRACTCE,AFFGEOID,GEOID,NAME,STUSPS,NAMELSADCO,STATE_NAME,LSAD,ALAND,AWATER,geometry,Census Tract ID
0,13,163,960300,1400000US13163960300,13163960300,9603,GA,Jefferson County,Georgia,CT,564728206,4845400,"{""type"": ""Polygon"", ""coordinates"": [[[-82.6619...",9603
1,37,109,071201,1400000US37109071201,37109071201,712.01,NC,Lincoln County,North Carolina,CT,5976613,6611763,"{""type"": ""Polygon"", ""coordinates"": [[[-80.9934...",712.01
2,25,025,050200,1400000US25025050200,25025050200,502,MA,Suffolk County,Massachusetts,CT,243271,0,"{""type"": ""Polygon"", ""coordinates"": [[[-71.0400...",502
3,17,031,230200,1400000US17031230200,17031230200,2302,IL,Cook County,Illinois,CT,214283,0,"{""type"": ""Polygon"", ""coordinates"": [[[-87.7134...",2302
4,41,047,001607,1400000US41047001607,41047001607,16.07,OR,Marion County,Oregon,CT,1814790,0,"{""type"": ""Polygon"", ""coordinates"": [[[-122.990...",16.07


In [11]:
# Function to extract coordinates from GeoJSON
def extract_coordinates(geometry):
    try:
        geojson_data = json.loads(geometry)
        coordinates = geojson_data['coordinates'][0]  # Assuming it's a Polygon with exterior ring
        return coordinates
    except json.JSONDecodeError:
        return None

In [12]:
# Apply the function to the 'geometry' column
df['coordinates'] = df['geometry'].apply(extract_coordinates)
df.shape
df.head()

# Administration.CensusTract: 
# CensusTractId, Name, GeoJson, IsActive, ModifyDate, ModifyUserId, CreateDate, CreateUserId

(85230, 15)

,STATEFP,COUNTYFP,TRACTCE,AFFGEOID,GEOID,NAME,STUSPS,NAMELSADCO,STATE_NAME,LSAD,ALAND,AWATER,geometry,Census Tract ID,coordinates
0,13,163,960300,1400000US13163960300,13163960300,9603,GA,Jefferson County,Georgia,CT,564728206,4845400,"{""type"": ""Polygon"", ""coordinates"": [[[-82.6619...",9603,"[[-82.66191700000009, 33.12633099913608, 0], [..."
1,37,109,071201,1400000US37109071201,37109071201,712.01,NC,Lincoln County,North Carolina,CT,5976613,6611763,"{""type"": ""Polygon"", ""coordinates"": [[[-80.9934...",712.01,"[[-80.99344400000007, 35.47878399910793, 0], [..."
2,25,025,050200,1400000US25025050200,25025050200,502,MA,Suffolk County,Massachusetts,CT,243271,0,"{""type"": ""Polygon"", ""coordinates"": [[[-71.0400...",502,"[[-71.04002900000008, 42.37864999906062, 0], [..."
3,17,031,230200,1400000US17031230200,17031230200,2302,IL,Cook County,Illinois,CT,214283,0,"{""type"": ""Polygon"", ""coordinates"": [[[-87.7134...",2302,"[[-87.7134170000001, 41.91358899906211, 0], [-..."
4,41,047,001607,1400000US41047001607,41047001607,16.07,OR,Marion County,Oregon,CT,1814790,0,"{""type"": ""Polygon"", ""coordinates"": [[[-122.990...",16.07,"[[-122.99031800000012, 44.96821399905682, 0], ..."


In [13]:
# extract the first values as the first coordinates 
df['FirstCoordinate'] = df['coordinates'].apply(lambda x: x[0] if isinstance(x, list) and len(x) > 0 else None)
df.head()

,STATEFP,COUNTYFP,TRACTCE,AFFGEOID,GEOID,NAME,STUSPS,NAMELSADCO,STATE_NAME,LSAD,ALAND,AWATER,geometry,Census Tract ID,coordinates,FirstCoordinate
0,13,163,960300,1400000US13163960300,13163960300,9603,GA,Jefferson County,Georgia,CT,564728206,4845400,"{""type"": ""Polygon"", ""coordinates"": [[[-82.6619...",9603,"[[-82.66191700000009, 33.12633099913608, 0], [...","[-82.66191700000009, 33.12633099913608, 0]"
1,37,109,071201,1400000US37109071201,37109071201,712.01,NC,Lincoln County,North Carolina,CT,5976613,6611763,"{""type"": ""Polygon"", ""coordinates"": [[[-80.9934...",712.01,"[[-80.99344400000007, 35.47878399910793, 0], [...","[-80.99344400000007, 35.47878399910793, 0]"
2,25,025,050200,1400000US25025050200,25025050200,502,MA,Suffolk County,Massachusetts,CT,243271,0,"{""type"": ""Polygon"", ""coordinates"": [[[-71.0400...",502,"[[-71.04002900000008, 42.37864999906062, 0], [...","[-71.04002900000008, 42.37864999906062, 0]"
3,17,031,230200,1400000US17031230200,17031230200,2302,IL,Cook County,Illinois,CT,214283,0,"{""type"": ""Polygon"", ""coordinates"": [[[-87.7134...",2302,"[[-87.7134170000001, 41.91358899906211, 0], [-...","[-87.7134170000001, 41.91358899906211, 0]"
4,41,047,001607,1400000US41047001607,41047001607,16.07,OR,Marion County,Oregon,CT,1814790,0,"{""type"": ""Polygon"", ""coordinates"": [[[-122.990...",16.07,"[[-122.99031800000012, 44.96821399905682, 0], ...","[-122.99031800000012, 44.96821399905682, 0]"


In [14]:
# separate the first two values from th coordinates column 
df[['Longitude', 'Latitude']] = df['FirstCoordinate'].apply(lambda x: pd.Series(x[:2]) if isinstance(x, list) and len(x) >= 2 else (None, None))
df.shape
df.head()

(85230, 18)

,STATEFP,COUNTYFP,TRACTCE,AFFGEOID,GEOID,NAME,STUSPS,NAMELSADCO,STATE_NAME,LSAD,ALAND,AWATER,geometry,Census Tract ID,coordinates,FirstCoordinate,Longitude,Latitude
0,13,163,960300,1400000US13163960300,13163960300,9603,GA,Jefferson County,Georgia,CT,564728206,4845400,"{""type"": ""Polygon"", ""coordinates"": [[[-82.6619...",9603,"[[-82.66191700000009, 33.12633099913608, 0], [...","[-82.66191700000009, 33.12633099913608, 0]",-82.661917,33.126331
1,37,109,071201,1400000US37109071201,37109071201,712.01,NC,Lincoln County,North Carolina,CT,5976613,6611763,"{""type"": ""Polygon"", ""coordinates"": [[[-80.9934...",712.01,"[[-80.99344400000007, 35.47878399910793, 0], [...","[-80.99344400000007, 35.47878399910793, 0]",-80.993444,35.478784
2,25,025,050200,1400000US25025050200,25025050200,502,MA,Suffolk County,Massachusetts,CT,243271,0,"{""type"": ""Polygon"", ""coordinates"": [[[-71.0400...",502,"[[-71.04002900000008, 42.37864999906062, 0], [...","[-71.04002900000008, 42.37864999906062, 0]",-71.040029,42.37865
3,17,031,230200,1400000US17031230200,17031230200,2302,IL,Cook County,Illinois,CT,214283,0,"{""type"": ""Polygon"", ""coordinates"": [[[-87.7134...",2302,"[[-87.7134170000001, 41.91358899906211, 0], [-...","[-87.7134170000001, 41.91358899906211, 0]",-87.713417,41.913589
4,41,047,001607,1400000US41047001607,41047001607,16.07,OR,Marion County,Oregon,CT,1814790,0,"{""type"": ""Polygon"", ""coordinates"": [[[-122.990...",16.07,"[[-122.99031800000012, 44.96821399905682, 0], ...","[-122.99031800000012, 44.96821399905682, 0]",-122.990318,44.968214


In [23]:
for col in df.columns:
    print(col)

STATEFP
COUNTYFP
TRACTCE
AFFGEOID
GEOID
NAME
STUSPS
NAMELSADCO
STATE_NAME
LSAD
ALAND
AWATER
geometry
Census Tract ID
coordinates
FirstCoordinate
Longitude
Latitude
coordinate1
coordinate2
coordinate3
coordinate4
coordinate5
coordinate6
coordinate7
coordinate8
coordinate9
coordinate10


In [29]:
df[['coordinate1', 'coordinate2', 'coordinate3', 'coordinate4', 'coordinate5',
    'coordinate6', 'coordinate7', 'coordinate8', 'coordinate9', 'coordinate10']] = \
    pd.DataFrame(df['coordinates'].apply(lambda x: x[:10] + [None] * (10 - len(x)) if isinstance(x, list) else [None]*10).tolist())

# Display the updated DataFrame
df.head()

,STATEFP,COUNTYFP,TRACTCE,AFFGEOID,GEOID,NAME,STUSPS,NAMELSADCO,STATE_NAME,LSAD,...,coordinate1,coordinate2,coordinate3,coordinate4,coordinate5,coordinate6,coordinate7,coordinate8,coordinate9,coordinate10
0,13,163,960300,1400000US13163960300,13163960300,9603,GA,Jefferson County,Georgia,CT,...,"[-82.66191700000009, 33.12633099913608, 0]","[-82.66192200000009, 33.12631099913608, 0]","[-82.66186700000009, 33.125144999136104, 0]","[-82.65916900000009, 33.123823999136114, 0]","[-82.65829300000009, 33.12211399913614, 0]","[-82.65859600000009, 33.11957399913617, 0]","[-82.65766300000008, 33.11815999913619, 0]","[-82.65561200000008, 33.1176939991362, 0]","[-82.65519600000009, 33.116085999136224, 0]","[-82.65211100000009, 33.115463999136246, 0]"
1,37,109,071201,1400000US37109071201,37109071201,712.01,NC,Lincoln County,North Carolina,CT,...,"[-80.99344400000007, 35.47878399910793, 0]","[-80.99385600000008, 35.47103499910802, 0]","[-80.99206200000009, 35.456004999108174, 0]","[-80.99211900000009, 35.4551029991082, 0]","[-80.98707400000009, 35.455762999108174, 0]","[-80.97550700000008, 35.456950999108166, 0]","[-80.97154200000008, 35.45565199910819, 0]","[-80.96740300000008, 35.45462799910819, 0]","[-80.96641900000009, 35.45386899910818, 0]","[-80.96541600000009, 35.45047199910823, 0]"
2,25,025,050200,1400000US25025050200,25025050200,502,MA,Suffolk County,Massachusetts,CT,...,"[-71.04002900000008, 42.37864999906062, 0]","[-71.03989900000008, 42.37711999906061, 0]","[-71.03997800000008, 42.374600999060625, 0]","[-71.03907500000007, 42.37416699906062, 0]","[-71.03878000000007, 42.37492699906062, 0]","[-71.03583400000007, 42.376030999060625, 0]","[-71.03625600000007, 42.37665199906063, 0]","[-71.03409600000008, 42.37744999906062, 0]","[-71.03367800000007, 42.376826999060626, 0]","[-71.03104400000007, 42.37731999906062, 0]"
3,17,031,230200,1400000US17031230200,17031230200,2302,IL,Cook County,Illinois,CT,...,"[-87.7134170000001, 41.91358899906211, 0]","[-87.71330500000009, 41.91003699906213, 0]","[-87.71190100000008, 41.91005899906212, 0]","[-87.70687700000009, 41.91014699906212, 0]","[-87.70676500000009, 41.910713999062125, 0]","[-87.70686000000009, 41.91371599906211, 0]","[-87.70855000000009, 41.91370499906211, 0]","[-87.7101800000001, 41.91362999906211, 0]","[-87.7134170000001, 41.91358899906211, 0]",None
4,41,047,001607,1400000US41047001607,41047001607,16.07,OR,Marion County,Oregon,CT,...,"[-122.99031800000012, 44.96821399905682, 0]","[-122.99030900000012, 44.95391999905681, 0]","[-122.98369000000011, 44.9538999990568, 0]","[-122.97698000000013, 44.95391999905681, 0]","[-122.97675000000012, 44.96202999905682, 0]","[-122.97670400000013, 44.965990999056814, 0]","[-122.97667000000011, 44.969859999056816, 0]","[-122.98347000000011, 44.969319999056815, 0]","[-122.98785300000011, 44.96862299905681, 0]","[-122.99031800000012, 44.96821399905682, 0]"


In [32]:
df_final = df[['STATEFP','COUNTYFP','TRACTCE','GEOID','Census Tract ID','STUSPS','NAMELSADCO','STATE_NAME','LSAD','FirstCoordinate','Latitude','Longitude','coordinate1',
'coordinate2',
'coordinate3',
'coordinate4',
'coordinate5',
'coordinate6',
'coordinate7',
'coordinate8',
'coordinate9',
'coordinate10']]
df_final.shape
df_final.head()

(85230, 22)

,STATEFP,COUNTYFP,TRACTCE,GEOID,Census Tract ID,STUSPS,NAMELSADCO,STATE_NAME,LSAD,FirstCoordinate,Latitude,Longitude,coordinate1,coordinate2,coordinate3,coordinate4,coordinate5,coordinate6,coordinate7,coordinate8,coordinate9,coordinate10
0,13,163,960300,13163960300,9603,GA,Jefferson County,Georgia,CT,"[-82.66191700000009, 33.12633099913608, 0]",33.126331,-82.661917,"[-82.66191700000009, 33.12633099913608, 0]","[-82.66192200000009, 33.12631099913608, 0]","[-82.66186700000009, 33.125144999136104, 0]","[-82.65916900000009, 33.123823999136114, 0]","[-82.65829300000009, 33.12211399913614, 0]","[-82.65859600000009, 33.11957399913617, 0]","[-82.65766300000008, 33.11815999913619, 0]","[-82.65561200000008, 33.1176939991362, 0]","[-82.65519600000009, 33.116085999136224, 0]","[-82.65211100000009, 33.115463999136246, 0]"
1,37,109,071201,37109071201,712.01,NC,Lincoln County,North Carolina,CT,"[-80.99344400000007, 35.47878399910793, 0]",35.478784,-80.993444,"[-80.99344400000007, 35.47878399910793, 0]","[-80.99385600000008, 35.47103499910802, 0]","[-80.99206200000009, 35.456004999108174, 0]","[-80.99211900000009, 35.4551029991082, 0]","[-80.98707400000009, 35.455762999108174, 0]","[-80.97550700000008, 35.456950999108166, 0]","[-80.97154200000008, 35.45565199910819, 0]","[-80.96740300000008, 35.45462799910819, 0]","[-80.96641900000009, 35.45386899910818, 0]","[-80.96541600000009, 35.45047199910823, 0]"
2,25,025,050200,25025050200,502,MA,Suffolk County,Massachusetts,CT,"[-71.04002900000008, 42.37864999906062, 0]",42.37865,-71.040029,"[-71.04002900000008, 42.37864999906062, 0]","[-71.03989900000008, 42.37711999906061, 0]","[-71.03997800000008, 42.374600999060625, 0]","[-71.03907500000007, 42.37416699906062, 0]","[-71.03878000000007, 42.37492699906062, 0]","[-71.03583400000007, 42.376030999060625, 0]","[-71.03625600000007, 42.37665199906063, 0]","[-71.03409600000008, 42.37744999906062, 0]","[-71.03367800000007, 42.376826999060626, 0]","[-71.03104400000007, 42.37731999906062, 0]"
3,17,031,230200,17031230200,2302,IL,Cook County,Illinois,CT,"[-87.7134170000001, 41.91358899906211, 0]",41.913589,-87.713417,"[-87.7134170000001, 41.91358899906211, 0]","[-87.71330500000009, 41.91003699906213, 0]","[-87.71190100000008, 41.91005899906212, 0]","[-87.70687700000009, 41.91014699906212, 0]","[-87.70676500000009, 41.910713999062125, 0]","[-87.70686000000009, 41.91371599906211, 0]","[-87.70855000000009, 41.91370499906211, 0]","[-87.7101800000001, 41.91362999906211, 0]","[-87.7134170000001, 41.91358899906211, 0]",None
4,41,047,001607,41047001607,16.07,OR,Marion County,Oregon,CT,"[-122.99031800000012, 44.96821399905682, 0]",44.968214,-122.990318,"[-122.99031800000012, 44.96821399905682, 0]","[-122.99030900000012, 44.95391999905681, 0]","[-122.98369000000011, 44.9538999990568, 0]","[-122.97698000000013, 44.95391999905681, 0]","[-122.97675000000012, 44.96202999905682, 0]","[-122.97670400000013, 44.965990999056814, 0]","[-122.97667000000011, 44.969859999056816, 0]","[-122.98347000000011, 44.969319999056815, 0]","[-122.98785300000011, 44.96862299905681, 0]","[-122.99031800000012, 44.96821399905682, 0]"


#### Importing the dataset to the SQL server

In [ ]:
pip install pandas pyodbc
pip install sqlalchemy

In [ ]:
import pandas as pd
import pyodbc

In [ ]:
# Replace these values with your SQL Server connection details
server = 'server_name'
database = 'database_name'
username = 'username'
password = 'password'

# Create a connection string
conn_str = f'DRIVER=SQL Server;SERVER={server};DATABASE={database};UID={username};PWD={password}'

# Establish a connection
conn = pyodbc.connect(conn_str)

In [ ]:
# Create a cursor
cursor = conn.cursor()

In [ ]:
table_name = 'geojson_table_name'

In [ ]:
# table creation 
# SQL command to create the table
create_table_sql = '''
CREATE TABLE {} (
    STATEFP INT,
    COUNTYFP INT,
    TRACTCE INT,
    AFFGEOID NVARCHAR(255),
    GEOID BIGINT,
    NAME NVARCHAR(255),
    NAMELSAD NVARCHAR(255),
    STUSPS NVARCHAR(255),
    NAMELSADCO NVARCHAR(255),
    STATE_NAME NVARCHAR(255),
    LSAD NVARCHAR(255),
    ALAND BIGINT,
    AWATER BIGINT,
    #geometry NVARCHAR(MAX),
    #coordinates NVARCHAR(MAX),
    latitude NVARCHAR(MAX),
    longitude NVARCHAR(MAX),
);
'''.format(table_name)

In [ ]:
# Execute to create the table
cursor.execute(create_table_sql)

In [ ]:
# Commit
conn.commit()

In [ ]:
# DataFrame to sqltable
df_final.to_sql('geojson_table_name', conn, index=False, if_exists='replace')